# mount google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# data

##loading data##

In [0]:
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import CountVectorizer

train_data = pd.read_csv('/content/drive/My Drive/SML_Project1/whodunnit/train_tweets.txt',names=['userid','tweet'],sep='\t',quoting=csv.QUOTE_NONE)
predict_data = pd.read_csv('/content/drive/My Drive/SML_Project1/whodunnit/test_tweets_unlabeled.txt',names=['tweet'],sep='\t',quoting=csv.QUOTE_NONE)

train_size = len(train_data)
all_tweets = pd.concat([train_data['tweet'],predict_data['tweet']],ignore_index = True)

count_vectorizer = CountVectorizer()
bow_data = count_vectorizer.fit_transform(all_tweets) #bag of word matrix
x_data = bow_data[:train_size]
x_predict = bow_data[train_size:]

# x_data = x_data.tocoo()
# print('x type: {}'.format(type(x_data)))
y_data_raw = np.array(train_data['userid'])


#labelling authorid as numbers from 0
num_classes = len(set(y_data_raw))
labels = np.arange(0,num_classes)

label_dict = dict(zip(list(set(y_data_raw)),labels))
y_data = []
for data in y_data_raw:
  y_data.append(label_dict[data])
y_data = np.array(y_data)


##split train and test data##

In [2]:
VALID_SIZE = 100
#split train and test data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data, test_size=0.1, random_state=1234)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)
x_validation = x_test[:VALID_SIZE]
y_validation = y_test[:VALID_SIZE]
print('x type: {}'.format(type(x_train)))
print('y type: {}'.format(type(y_train)))
print('num_classes {}'.format(num_classes))

(296038, 252152) (296038,)
(32894, 252152) (32894,)
x type: <class 'scipy.sparse.csr.csr_matrix'>
y type: <class 'numpy.ndarray'>
num_classes 9297


##rongxiao Data

In [1]:
from scipy import sparse
import numpy as np
x_train = sparse.load_npz('/content/drive/My Drive/SML_Project1/rongxiaoData/X_train.npz')
x_test = sparse.load_npz('/content/drive/My Drive/SML_Project1/rongxiaoData/X_test.npz')
x_predict = sparse.load_npz('/content/drive/My Drive/SML_Project1/rongxiaoData/X_predict.npz')
y_train = np.load('/content/drive/My Drive/SML_Project1/rongxiaoData/Y_train.npz')
y_test = np.load('/content/drive/My Drive/SML_Project1/rongxiaoData/Y_test.npz')
# print(y_train.files)
y_train = y_train['array']
y_test = y_test['array']
print(x_test.shape)
num_classes = x_train.shape[1]

                          

(32894, 150105)


# **sklearn network**

In [0]:
#network
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam',alpha=1e-5,hidden_layer_sizes=(100,100),batch_size=100)
clf.fit(x_train,y_train)
results = clf.predict(x_test)
# print(results)
# print(y_test)
print(len(y_test))
accuracy = clf.score(x_test,y_test)
print(accuracy)
right_test = 0
for i,result in enumerate(results):
  if result == y_test[i]:
    right_test += 1
print(right_test/len(y_test))

2000
0.027
0.027


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


# pytorch network

##csr to sparse tensor##

In [0]:
import torch
from scipy import sparse
def csr2sparseTensor(x_csr):
  x_coo = x_csr.tocoo()
  data_shape = x_coo.shape
  data_value = x_coo.data
  data_index = np.vstack((x_coo.row,x_coo.col)) #(row[i],col[i])= data[i]
  tensor_index = torch.LongTensor(data_index)
  tensor_value = torch.FloatTensor(data_value)
  #create a tensor from a saving space format
  x_tensor = torch.sparse.FloatTensor(tensor_index,tensor_value,torch.Size(data_shape)) #.to_dense() can give the full matrix
  return x_tensor

# A = np.array([[0,1,1],[1,0,0]])
# sA = sparse.csr_matrix(A)
# tA = csr2sparseTensor(sA)
# print(sA[0])
# for t in sA:
#   print(t)

##CPU version##

In [0]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from datetime import datetime
HIDDEN_UNITS = 10000
LR = 0.01
EPOCHES = 300

class MY_CLASSIFIER(torch.nn.Module):
  def __init__(self,n_feature,n_hidden,n_output):
    super(MY_CLASSIFIER,self).__init__()
    self.h1 = torch.nn.Linear(n_feature,n_hidden)
    self.output = torch.nn.Linear(n_hidden,n_output)
    
  def forward(self,x):
    x = F.relu(self.h1(x))
    x = self.output(x)
    return x
  

def train(x_train,y_train,num_classes):
  x_train_v = Variable(csr2sparseTensor(x_train),requires_grad=True)
  y_train_tensor = torch.LongTensor(y_train)
  
  my_net = MY_CLASSIFIER(x_train.shape[1],HIDDEN_UNITS,num_classes)
  #print(my_net)
  #optimizer = torch.optim.Adam(my_net.parameters(),lr = LR, betas = (0.9,0.99))
  optimizer = torch.optim.SGD(my_net.parameters(),lr = LR)
  loss_func = torch.nn.CrossEntropyLoss()
  for epoch in range(EPOCHES):
    output = my_net(x_train_v)
    loss = loss_func(output,y_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  torch.save(my_net.state_dict(),'my_net.pkl')
  

def predict(x_test,num_classes):
  x_test_tensor = torch.FloatTensor(x_test)
  my_net = MY_CLASSIFIER(x_test.shape[1],HIDDEN_UNITS,num_classes)
  my_net.load_state_dict(torch.load('my_net.pkl'))
  predictions = my_net(x_test_tensor)
#   print(predictions.shape)
#   print(y_test.shape)
  return predictions

def accuracy_compution(y_test,predictions):
  #print(F.softmax(predictions)[0])
  classifications = torch.max(F.softmax(predictions),1)[1]
  #print(classifications)
  #print(y_test)
  correct_num = 0
  for i,y in enumerate(y_test):
    if classifications[i] == y:
      correct_num += 1
  return correct_num/len(y_test)
    
startTime = datetime.now()
train(x_train.toarray(),y_train,num_classes)
predictions = predict(x_test.toarray(),num_classes)
accuracy = accuracy_compution(y_test,predictions)
print(accuracy)
endTime = datetime.now()
print('time: {}'.format(endTime-startTime))

0.02
time: 0:01:15.485052


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


##GPU version##

In [25]:
!/opt/bin/nvidia-smi

Sat Sep  7 14:11:29 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    87W / 149W |  11310MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [26]:
!ps -aux|grep python

root          22  0.2  0.3 476636 99308 ?        Sl   13:52   0:02 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
root         141  4.1 13.8 55803816 3696076 ?    Ssl  13:52   0:46 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-0548e351-da55-48f2-bbc3-11eeb4d4e068.json
root         619  0.0  0.0  39192  6628 ?        S    14:11   0:00 /bin/bash -c ps -aux|grep python
root         621  0.0  0.0  38568  4992 ?        S    14:11   0:00 grep python


In [0]:
!kill 141

In [0]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import TensorDataset,DataLoader
from datetime import datetime
EPOCHES = 5
HIDDEN_UNITS = 1000
LR = 0.01
BATCH_SIZE = 1000
OUTPUT_MODEL_STEP = 1
file_dir = '/content/drive/My Drive/SML_Project1/rongxiaoData/'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class MY_CLASSIFIER(torch.nn.Module):
  def __init__(self,n_feature,n_hidden,n_output):
    super(MY_CLASSIFIER,self).__init__()
    self.h1 = torch.nn.Linear(n_feature,n_hidden)
    self.h2 = torch.nn.Linear(n_hidden,n_hidden)
    self.output = torch.nn.Linear(n_hidden,n_output)
    
#     print(self.h1)
#     print(self.output)
    
  def forward(self,x):
    x = F.relu(self.h1(x))
    x = F.relu(self.h2(x))
    x = self.output(x)
    return x
  

def train(x_train,y_train,num_classes,start_epoch):  
  #batch training
  #TensorDataset(x,y):x cannot be csr matrix
#   train_dataset = TensorDataset(x_train, y_train)
  
#   loader = DataLoader(
#       dataset=train_dataset,
#       batch_size=BATCH_SIZE,
#       num_workers=0
#   )
  
  torch.cuda.empty_cache() #release useless gpu memory
  start_train = datetime.now()
  num_features = x_train.shape[1]
  my_net = MY_CLASSIFIER(num_features,HIDDEN_UNITS,num_classes).to(device)
  if start_epoch > 0:
    model_file_name = file_dir + 'my_net_'+str(start_epoch)+'.pkl'
    my_net.load_state_dict(torch.load(model_file_name))
  total_paras = sum(p.numel() for p in my_net.parameters())
  total_paras_trainable = sum(p.numel() for p in my_net.parameters() if p.requires_grad)
  print('total_paras:{}\ntotal_paras_trainable:{}'.format(total_paras,total_paras_trainable))
  #print(my_net)
   
  optimizer = torch.optim.Adam(my_net.parameters(),lr = LR, betas = (0.9,0.99))
  #optimizer = torch.optim.SGD(my_net.parameters(),lr = LR)
  loss_func = torch.nn.CrossEntropyLoss()
  
  train_data_size = len(y_train)
  for epoch in range(start_epoch,EPOCHES):
    time_cost = datetime.now() - start_train
    print('epoch {} has started; time_cost_sofar:'.format(epoch,time_cost))
    for idx in range(0,train_data_size,BATCH_SIZE):
      if idx % 100000 == 0:
        time_cost = datetime.now() - start_train
        print("epoch: {} and idx: {} ; time_cost_sofar: {}".format(epoch,idx,time_cost))
      x_batch = x_train[idx:idx+BATCH_SIZE]
      y_batch = y_train[idx:idx+BATCH_SIZE]
      
      #convert each batch of data to tensor and passing it to gpu step by step
      x_train_batch = Variable(csr2sparseTensor(x_batch),requires_grad=True)
      y_train_batch = torch.LongTensor(y_batch).to(device) #32 bits
      
      #can be sparse on gpu ,but when feed to model have to be dense
      #only the data which would be computed on gpu is passing to gpu exclouding x_train_batch
      output = my_net(x_train_batch.to_dense().to(device))
      loss = loss_func(output,y_train_batch)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    if (epoch+1) % OUTPUT_MODEL_STEP == 0 or epoch == 0:
      model_file_name = file_dir + '/my_net_'+str(epoch+1)+'.pkl'
      torch.save(my_net.state_dict(),model_file_name)
      time_cost = datetime.now() - start_train
      print("epoch: {} has finished; time_cost_sofar: {}".format(epoch,idx,time_cost))


def predict(x_test,num_classes,epoch):
  
  my_net = MY_CLASSIFIER(x_test.shape[1],HIDDEN_UNITS,num_classes).to(device)
  if epoch >= 0:
    model_file_name = file_dir + '/my_net_'+str(epoch)+'.pkl'
    my_net.load_state_dict(torch.load(model_file_name))
  
  #predict in small pieces
  test_data_size = x_test.shape[0]
  predictions = np.array([])
  for idx in range(0,test_data_size,BATCH_SIZE):
    x_batch = x_test[idx:idx+BATCH_SIZE]
    x_test_batch = csr2sparseTensor(x_batch)
    batch_predictions = my_net(x_test_batch.to_dense().to(device))
    batch_classifications = torch.max(F.softmax(batch_predictions),1)[1].cpu()
    predictions = np.hstack((predictions,batch_classifications.numpy()))
#   print(predictions.shape)
#   print(y_test.shape)
  return predictions

def accuracy_compution(y_test,predictions):
  correct_num = 0
  for i,y in enumerate(y_test):
    if predictions[i] == y:
      correct_num += 1
  return correct_num/len(y_test)
    
startTime = datetime.now()
print(device)
gpu_memory = torch.cuda.max_memory_allocated(device)
print('gpu memory: {} GB'.format(gpu_memory/(1024**3)))
torch.cuda.empty_cache()
gpu_memory_allocated = torch.cuda.memory_allocated(device)
print('gpu memory allocated: {} GB'.format(gpu_memory_allocated/(1024**3)))

train(x_train,y_train,num_classes,0)
epoch_list = [1,2,3]
for epoch in epoch_list:
  predictions = predict(x_test,num_classes,epoch)
  print(predictions.shape)
  accuracy = accuracy_compution(y_test,predictions)
  print("epoch: {}  accuracy: {}".format(epoch,accuracy))
  endTime = datetime.now()
  print('time: {}'.format(endTime-startTime))
#pytorch GPU 7 mins for each epoch
#pytorch CPU 45 mins for each epoch
#1 layer-1000-units test accuracy:[1,17.35%],[2,17.79%][3,17.03%] 跑30没结果？？？？
#2 layer-1000-units test accuracy:[1,11.21%],[2,13.12%],[3,13.03%]
#1 layer-500-units test accuracy:[1,10.94%],[2,13.4%],[3,12.92%]
#new features

cuda:0
gpu memory: 0.0 GB
gpu memory allocated: 0.0 GB
total_paras:301362105
total_paras_trainable:301362105
epoch 0 has started; time_cost_sofar:
epoch: 0 and idx: 0 ; time_cost_sofar: 0:00:05.454188


## do predict on the unlabelled data

In [30]:
import pandas as pd
import numpy as np
import csv
re_label_dict = dict(zip(label_dict.values(),label_dict.keys()))
print(re_label_dict)
predictions = predict(x_predict,num_classes,2)
prediction_size = len(predictions)
result_labels = np.zeros((prediction_size,2),dtype=np.int32)
for i,prediction in enumerate(predictions):
  result_labels[i] = np.array([i+1,re_label_dict[prediction]])
# print(result_labels)
result = pd.DataFrame({"Id": result_labels[:,0],'Predicted':result_labels[:,1]})
result.to_csv(file_dir + "/predictions.csv",index=False)

{0: 2, 1: 3, 2: 4, 3: 6, 4: 7, 5: 8, 6: 9, 7: 10, 8: 11, 9: 12, 10: 13, 11: 14, 12: 15, 13: 16, 14: 17, 15: 18, 16: 19, 17: 20, 18: 21, 19: 22, 20: 23, 21: 24, 22: 25, 23: 26, 24: 27, 25: 28, 26: 30, 27: 31, 28: 34, 29: 35, 30: 36, 31: 38, 32: 39, 33: 41, 34: 42, 35: 43, 36: 44, 37: 45, 38: 46, 39: 47, 40: 48, 41: 49, 42: 50, 43: 51, 44: 52, 45: 53, 46: 54, 47: 55, 48: 56, 49: 57, 50: 58, 51: 59, 52: 60, 53: 61, 54: 62, 55: 63, 56: 65, 57: 66, 58: 67, 59: 68, 60: 69, 61: 70, 62: 71, 63: 72, 64: 73, 65: 74, 66: 75, 67: 76, 68: 77, 69: 78, 70: 79, 71: 80, 72: 81, 73: 82, 74: 83, 75: 84, 76: 85, 77: 86, 78: 87, 79: 89, 80: 90, 81: 91, 82: 92, 83: 93, 84: 94, 85: 95, 86: 96, 87: 97, 88: 99, 89: 100, 90: 101, 91: 102, 92: 103, 93: 104, 94: 105, 95: 106, 96: 107, 97: 109, 98: 111, 99: 112, 100: 113, 101: 114, 102: 115, 103: 116, 104: 117, 105: 118, 106: 119, 107: 120, 108: 121, 109: 122, 110: 124, 111: 126, 112: 127, 113: 128, 114: 129, 115: 130, 116: 132, 117: 133, 118: 134, 119: 136, 120: 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:97: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


TypeError: ignored

# Tensorflow network TPU

In [3]:
from google.colab import auth
auth.authenticate_user()

W0906 04:40:49.577821 140616493561728 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
import tensorflow as tf
import numpy as np
import os
from google.colab import files
from datetime import datetime
HIDDEN = 1000
EPOCHES = 1000
BATCH_SIZE = 1000
#setup TPU
# TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

def add_layer(input,in_size,out_size,acctivation = None):
  Weight = tf.Variable(tf.random.normal([in_size,out_size]),dtype=tf.float32)
  biase = tf.zeros([1,out_size],dtype=tf.float32) + 0.5
  w_plus_biase = tf.matmul(input,Weight) + biase
  if acctivation is None:
    output = w_plus_biase
  else:
    output = acctivation(w_plus_biase)
  return output

def list_one_hot(list,num_classes):
  list = list.astype(np.int)
  list_size = len(list)
  list_results = np.zeros((list_size,num_classes))
  list_results[np.arange(list_size),list] = 1
  return list_results

def compute_accuracy(x,y):
  y_prediction = sess.run(prediction,feed_dict={xs:x,ys:y})
  equal_list = tf.equal(tf.argmax(y,1),tf.argmax(y_prediction,1))
  accuracy = tf.reduce_mean(tf.cast(equal_list,tf.float32))
  result = sess.run(accuracy,feed_dict={xs:x,ys:y})
  return result

train_size = x_train.shape[0]
vocab_size = x_train.shape[1] 
xs = tf.placeholder(tf.float32,[None,vocab_size])
ys = tf.placeholder(tf.float32,[None,num_classes])

h1 = add_layer(xs,vocab_size,HIDDEN,acctivation = tf.nn.relu)
prediction = add_layer(h1,HIDDEN,num_classes,acctivation = tf.nn.softmax)

cross_entropy = tf.reduce_mean(-ys*tf.log(tf.clip_by_value(prediction,1e-10,1.0)))
train = tf.train.AdamOptimizer(0.5,0.9,0.99).minimize(cross_entropy)
y_valid = list_one_hot(y_validation,num_classes)
# print(y_validation.shape)
saver = tf.train.Saver()
x_valid = x_validation.toarray()

with tf.Session() as sess:
  startTime = datetime.now()
  sess.run(tf.global_variables_initializer())
  for epoch in range(1,EPOCHES+1):
    endTime = datetime.now()
    print('epoch {} started; time_cost_so_far: {}'.format(epoch,(endTime-startTime)))
    for idx in range(0,train_size,BATCH_SIZE):
      x_batch = x_train[idx:idx+BATCH_SIZE].toarray()
      y_batch = list_one_hot(y_train[idx:idx+BATCH_SIZE],num_classes)     
      sess.run(train,feed_dict = {xs:x_batch, ys:y_batch})
      if idx % 50000 == 0:
        endTime = datetime.now()
        print('epoch:{}   idx:{}   time_cost_so_far: {}'.format(epoch,idx,(endTime-startTime)))
        
    if epoch % 200 == 0:
      loss = sess.run(cross_entropy,feed_dict = {xs:x_batch,ys:y_batch})
      acc = compute_accuracy(x_valid,y_valid)
      print('epoch: {} --> loss:{}  --> acc :{}'.format(epoch,loss,acc))
      filename = '/content/drive/My Drive/SML/CPU/my_model_'+str(epoch)+'.ckpt'
      saver.save(sess,filename)
#tensorflow TPU 25 mins for each epoch; maybe is cpu
#tensorflow CPU 25 mins for each epoch;
#tensorflow GPU 9 mins for each epoch;

# Test

In [0]:
import torch
A = torch.LongTensor([[1,1,1],[1,1,1]])
B = torch.LongTensor([[2,2,2],[2,2,2]])
print(A+B)

tensor([[3, 3, 3],
        [3, 3, 3]])


In [0]:
from scipy import sparse 
A = np.array([[1,0,3],[0,1,0]])
sA = sparse.csr_matrix(A)
B = np.array([[1],[0]])
print(sA)
print(B)
data = zip(sA,B)
for i in data:
  print("i : {}".format(i[0]))

In [0]:
A = np.array([1,2,3,4]).reshape(2,2)
B = sparse.csr_matrix(A)
print(A[2:3])
# multiple = 12//3
# x = 9# + 3
# for i in range(0,x,3):
#   print(i)

In [0]:
a = np.array([2,3])
b = torch.LongTensor([1,2])
c = np.hstack((a,b.numpy()))
print(c)

In [0]:
for i in range(20,30):
  print(i)

In [0]:
import numpy as np
print(np.zeros([3,3]))
print(np.arange(10))

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0 1 2 3 4 5 6 7 8 9]


In [0]:
import os
tpu = 'grpc://'+os.environ['COLAB_TPU_ADDR']
!echo $TPU_NAME

grpc://10.47.16.74:8470


In [0]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

In [0]:
# from scipy import sparse
# import tensorflow as tf
# import numpy as np
# import os

# def csr2tfSparse(csr_m):
#   coo_m = csr_m.tocoo()
#   values = coo_m.data.astype(np.float32)
#   the_shape = coo_m.shape
#   idxs = np.hstack((coo_m.row[:,np.newaxis],coo_m.col[:,np.newaxis]))  #[row[i],col[i]] = data[i]
#   tf_sparse_m = tf.sparse.SparseTensor(idxs,values,the_shape)
#   return tf_sparse_m
  
# a = np.array([[1,2,3],[0,3,0]])
# a_csr = sparse.csr_matrix([[1,2,3],[0,3,0]])

# a_tf = csr2tfSparse(a_csr)
# b = np.random.randint(20,size=(3,2))

# a = tf.convert_to_tensor(a,dtype=tf.float32)
# b = tf.convert_to_tensor(b,dtype=tf.float32)
# a_b = tf.matmul(a,b)
# a_tf_b = tf.sparse.sparse_dense_matmul(a_tf,b) #TPU cannot do this
# with tf.Session(TPU_NAME) as sess:
#   print(sess.run(a_b))
#   print(sess.run(a_tf_b))